# BETA_random_eqrthquake_locations_w_pyvista_vtk

Testing the random picking of subsurface eqrthquake locations (moment-tensor locations) and 3D plotting of the picks and model 

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy

from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 

from sys import argv

from matplotlib.colors import Normalize
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import compress_gm3d_to_file
from pyaspect.model.gm3d_utils import decompress_gm3d_from_file
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import StationHeader
from pyaspect.specfemio.headers import SolutionHeader
from pyaspect.specfemio.headers import CMTSolutionHeader
from pyaspect.specfemio.headers import ForceSolutionHeader
from pyaspect.specfemio.write import write_cmtsolution
from pyaspect.specfemio.write import write_forcesolution
from pyaspect.specfemio.write import write_grouped_forcesolutions
from pyaspect.specfemio.write import write_stations
from pyaspect.specfemio.read import read_stations
from pyaspect.specfemio.read import read_solution
from pyaspect.specfemio.read import read_cmtsolution
from pyaspect.specfemio.read import read_forcesolution
from pyaspect.specfemio.utils import *

### Step 1 

Extract the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_in_dir} 

### Step 6 

Decompress the ndarray of the sliced, subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# set filename then used it to decompress model
ifqn = f'{data_out_dir}/vsliced_subsmp_smth_nam_2017_vp_vs_rho_Q_model_dx100_dy100_dz100_maxdepth5850_sig250.npz'
vslice_gm3d, other_pars = decompress_gm3d_from_file(ifqn)

print()
print('decompressed gridded model\n:',vslice_gm3d) 
print()
print('other parameters:\n',other_pars)
print()

# WARNING: this will unpack all other_pars, if you overwrite a variable of the samename as val(key), then you 
#          may not notice, and this may cause large headaches.  I use it because I am aware of it.
'''
for key in other_pars:
    locals()[key] = other_pars[key]  #this is more advanced python than I think is reasonable for most 
sig_meters = sig
''';

# another way to get these varibles is just use the accessor functions for the gridmod3d.  We need them later.
xmin = other_pars['xmin']
dx   = other_pars['dx']
nx   = other_pars['nx']
ymin = other_pars['ymin']
dy   = other_pars['dy']
ny   = other_pars['ny']
zmin = other_pars['zmin']
dz   = other_pars['dz']
nz   = other_pars['nz']
sig_meters = other_pars['sig']  # this variable is used later
print('sig_meters:',sig_meters)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
nam_dims = list(vslice_gm3d.get_npoints())
nam_origin = list(vslice_gm3d.get_gorigin())
nam_origin[2] *= -1
nam_origin = tuple(nam_origin)
nam_spacing = list(vslice_gm3d.get_deltas())
nam_spacing[2] *=-1
nam_spacing = tuple(nam_spacing)
print('nam_dims:',nam_dims)
print('nam_origin:',nam_origin)
print('nam_spacing:',nam_spacing)

# Edit the spatial reference
grid.dimensions = np.array(nam_dims) + 1
grid.origin = nam_origin  # The bottom left corner of the data set
grid.spacing = nam_spacing  # These are the cell sizes along each axis
nam_pvalues = vslice_gm3d.getNPArray()[0]
print('pvalues.shape:',nam_pvalues.shape)

# Add the data values to the cell data
grid.cell_arrays["values"] = nam_pvalues.flatten(order="F")  # Flatten the array!

# Now plot the grid!
cmap = plt.cm.jet
#grid.plot(show_edges=True,cmap=cmap)
grid.plot(cmap=cmap,opacity=1.0)


In [ ]:
slices = grid.slice_orthogonal()

#slices.plot(show_edges=True,cmap=cmap)
slices.plot(cmap=cmap)

## create random virtual source (to specfem stations, but using reciprocity -- sources)

In [ ]:
coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


n_rand_p = 1000

lrx = np.min(xc)
lry = np.min(yc)
lrz = -4300.0

hrx = np.max(xc)
hry = np.max(yc)
hrz = -3000.0

srx = hrx - lrx
sry = hry - lry
srz = hrz - lrz

r_xyz_list = []
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    rz = lrz + srz*np.random.rand()
    r_xyz_list.append([rx,ry,rz])
    
r_xyz = np.array(r_xyz_list)
    

#r_xyz = np.vstack(np.meshgrid(rx,ry,rz)).reshape(3,-1).T
print('r_xyz:\n',r_xyz)


In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
!pwd

In [ ]:
#!mkdir {data_out_dir}/tmp
!ls {data_out_dir}

## make cross (half or full) group for calculating spacial derivatives

In [ ]:
if 'l_grp_stations' in locals() or 'l_grp_stations' in globals():
    print('deleting')
    del l_grp_stations
    
# make a list of station headers for each random location
l_stations = []
for i in range(len(r_xyz)):
    
    name = 't' + str(i).zfill(len(str(len(r_xyz))))
    new_s = StationHeader(name=name,
                          network='NL',
                          lat_yc=r_xyz[i,1],
                          lon_xc=r_xyz[i,0],
                          elevation=0.0,
                          depth=r_xyz[i,2],
                          trid=i)
    l_stations.append(new_s)
#print('len(l_stats):',len(l_stations))
                                           
# make the group membors for each station above
# function below returns a list[list[]] like structure
l_grp_stations = make_grouped_half_cross_station_headers(stations=l_stations,delta=250.0)

# this is a list[] structure with unique stations (small chance a group member has same coordinate)
s_grp_stations = sorted(copy.deepcopy(flatten_grouped_headers_unique(l_grp_stations)))

# if lengths are the same then all group members are unique
print(f'len(l_grp): {len(flatten_grouped_headers(l_grp_stations))}')
print(f'len(s_grp): {len(s_grp_stations)}')

#flatten the l_grp_stations (NOT guarantied unique!)
l_grp_stations = sorted(flatten_grouped_headers(l_grp_stations))

In [ ]:
#write_stations(data_out_dir + '/tmp',l_grp_stations,fname='STATIONS_GROUPED')

# NOT grouped -> list[] instead of list[list[]]
write_stations(data_out_dir + '/tmp',l_grp_stations)

# For records only!
#write_stations(data_out_dir + '/tmp',l_grp_stations,write_h=False) 

In [ ]:
!ls -ltrh {data_out_dir + '/tmp'} 

In [ ]:
!head {data_out_dir}/tmp/STATIONS

In [ ]:
!tail {data_out_dir}/tmp/STATIONS

In [ ]:
fqp = data_out_dir + '/tmp'
rw_stations = sorted(read_stations(fqp))

print('stations equal?:', rw_stations == l_grp_stations)

In [ ]:
all_g_xyz = get_xyz_coords_from_station_list(rw_stations)
pv_all_points = pv.wrap(all_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_all_points, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## make random virtual recievers locations (solutions/sources in specfem)

In [ ]:
coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


n_rand_p = 5
rz = 150

lrx = np.min(xc)
lry = np.min(yc)

hrx = np.max(xc)
hry = np.max(yc)

srx = hrx - lrx
sry = hry - lry

s_xyz_list = []
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    s_xyz_list.append([rx,ry,rz])
    
s_xyz = np.array(s_xyz_list)
    

print('s_xyz:\n',s_xyz)


## make force solution headers

In [ ]:
# make a list of station headers for each random location
l_solutions = []
for i in range(len(s_xyz)):
    
    name = 't' + str(i).zfill(len(str(len(r_xyz))))
    new_s = StationHeader(name=name,
                          network='NL',
                          lat_yc=r_xyz[i,1],
                          lon_xc=r_xyz[i,0],
                          elevation=0.0,
                          depth=r_xyz[i,2],
                          trid=i)
    new_s = ForceSolutionHeader(ename=f'Event-{str(i).zfill(4)}',
                                lat_yc=s_xyz[i,1],
                                lon_xc=s_xyz[i,0],
                                depth=s_xyz[i,2],
                                tshift=0.0,
                                date=datetime.datetime.now(),
                                f0=0.0,
                                factor_fs=1,
                                comp_src_EX=1,
                                comp_src_NY=0,
                                comp_src_Zup=0,
                                proj_id=0,
                                eid=i,
                                sid=0)
    l_solutions.append(new_s)

## make solution group

In [ ]:
l_grp_solutions_h = make_grouped_triplet_force_solution_headers(solutions=l_solutions)
grp_s_xyz = get_xyz_coords_from_solution_list(flatten_grouped_headers(l_grp_solutions_h))

for s in flatten_grouped_headers(l_grp_solutions_h):
    print(f'solution:\n{s}')
    print()
print(f'coords:\n{grp_s_xyz}')
print()

In [ ]:
for grp_s in l_grp_solutions_h:
    p = f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}'
    #print(p)
    Path(p).mkdir(parents=True, exist_ok=True)

In [ ]:
!ls {data_out_dir}/tmp
!pwd

In [ ]:
for grp_s in l_grp_solutions_h:
    write_grouped_forcesolutions(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}',grp_s)
    #write_grouped_forcesolutions(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}',grp_s,write_h=False)
    #print(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}')
    #for s in grp_s:
        #print(f's:\n{s}')


In [ ]:
!ls -lrth {data_out_dir}/tmp/run*

In [ ]:
x = goobledoobleydoo('42')

In [ ]:
!ls -ltrh {data_out_dir + '/tmp/run*'} 

In [ ]:
!cat {data_out_dir}/tmp/CMTSOLUTION

In [ ]:
!cat {data_out_dir}/tmp/FORCESOLUTION

In [ ]:
fqp = f'{data_out_dir}/tmp'
read_gen_cmt = read_solution(fqp,'CMTSOLUTION')
read_gen_fs  = read_solution(fqp,'FORCESOLUTION')
read_cmt = read_cmtsolution(fqp,fname='CMTSOLUTION')
read_fs  = read_forcesolution(fqp,fname='FORCESOLUTION')
    
print('gen cmt:\n',read_gen_cmt)
print()
print('gen fs:\n',read_gen_fs)
print()
print('cmt:\n',read_cmt)
print()
print('fs:\n',read_fs)
print()

In [ ]:
all_g_xyz = get_xyz_coords_from_station_list(rw_stations)
exc_g_xyz = get_xyz_coords_from_station_list_except(rw_stations,key='gid',val=0)

In [ ]:
pv_all_points = pv.wrap(all_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_all_points, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
pv_exc_points = pv.wrap(exc_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_exc_points, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
from pyaspect.specfemio.headers import RecordHeader

class StationTrace(object):
    
    def __init__(self, header=None, data=None, start_t=0, dt=1):
        
        if not isinstance(header,StationHeader):
            raise Exception('arg: \'header\' must be of type StationHeader')
            
        self.header  = copy.deepcopy(header)
        self.data    = np.array(data,dtype=np.float32)
        self.start_t = np.float32(start_t)
        self.dt      = np.float32(dt)
        
        
    def __getitem__(self, islice):
        return self.data[islice]
    
    def __setitem__(self, islice, value):
        self.data[islice] = value
        
    def __str__(self):
        out_str  = f'Station Header:\n{self.header}\n'
        out_str += f'Data:\n{self.data}'
        return out_str
    
    def __repr__(self):
        out_str  = f'Station Header:\n{self.header.__repr__()}\n'
        out_str += f'Data:\n{self.data.__repr__()}'
        return out_str
    
    def times(self):
        return self.start_t + np.arange(len(self.data),dtype=np.float32)*self.dt
        
        
    
df_force_h0 = test_force_h.copy()
df_force_h1 = test_force_h.copy()
df_force_h2 = test_force_h.copy()

print('h0 == h1:',df_force_h0 == df_force_h1)
print()
print('h0 == h2:',df_force_h0 == df_force_h2)
print()

df_force_h0.sid = 0
df_force_h0.eid = 0
df_force_h1.sid = 1
df_force_h1.eid = 0
df_force_h2.sid = 2
df_force_h2.eid = 0
print('df_force_h0.sid:',df_force_h0.sid)
print()
print('after h0 == h1:',df_force_h0 == df_force_h1)
print()
print('after h0 == h2:',df_force_h0 == df_force_h2)
print()

flat_grp_stations = sorted(flatten_grouped_headers(l_grp_stations))
print(f'flat_grp_stations[0]:\n{flat_grp_stations[0]}')
print()
#t_record = RecordHeader(solutions_h=[df_force_h0],stations_h=flat_grp_stations)
t_record = RecordHeader(solutions_h=[df_force_h0,df_force_h1,df_force_h2],stations_h=flatten_grouped_headers(l_grp_stations))
print(f'type(df_force_h0)={type(df_force_h0)}\n')

print(t_record)
print()

rec_solutions_h = t_record.get_solutions_header_list()[0]
rec_stations_h  = t_record.get_stations_header_list()
print(f'station[0]:\n{rec_stations_h[0]}')
print(f'isinstance(rec_stations_h[0],ForceSolutionHeader)={isinstance(rec_stations_h[0],StationHeader)}\n')
print(f'type(flat_grp_stations[0])==type(rec_stations_h[0]) = {type(flat_grp_stations[0])==type(rec_stations_h[0])}\n')
print()
print(f'solution[0]:\n{rec_solutions_h}')
print(f'isinstance(rec_solutions_h,ForceSolutionHeader)={isinstance(rec_solutions_h,ForceSolutionHeader)}\n')
print(f'type(df_force_h0)==type(rec_solutions_h) = {type(df_force_h0)==type(rec_solutions_h)}\n')
print()

h_vals = []
#key = 'new_eid'
#for s in rec_stations_h:
    #h_vals.append(s.trid)
    
src = rec_solutions_h
key = 'offset'
for s in rec_stations_h:
    x_sqrd = (src.lon_xc - s.lon_xc)**2
    y_sqrd = (src.lat_yc - s.lat_yc)**2
    offset = np.sqrt(x_sqrd + y_sqrd)
    h_vals.append(offset)
    
t_record.add_station_header_word(key,h_vals)
#print('added header word:\n',t_record)
print('added header word:\n',t_record.stations_df[:21])
print()
        
'''
    
t_record.add_header_word_to_stations(offset_func)

print(t_record)
print()
print('added header words:\n',t_record.added_header_words)
print()
print('Traces:\n',t_record.traces)
print()
''';

'''
t_st = StationTrace(s_grp_stations[0],np.arange(2000), start_t=0, dt=0.25)

print(t_st)
print()

t_st[1:3] = -1*np.ones((2))
print('slice-data:',t_st[:5])
print()

print('times:',t_st.times())
''';

In [ ]:
f = open(data_out_dir + '/test_record_header.pickle','wb')
pickle.dump(t_record,f)
f.close()

In [ ]:
!ls -lth {data_out_dir + '/test_record_header.pickle'} 

In [ ]:
f = open(data_out_dir + '/test_record_header.pickle','rb')
dill_t_record = pickle.load(f)
f.close()

print(dill_t_record)
print()

print('slice:\n',type(dill_t_record.stations_df.iloc[0]))
print()
print('to_dict():\n',dill_t_record.stations_df.iloc[0].to_dict())
print()
print('stations equal?:',dill_t_record.stations_df.equals(t_record.stations_df))
print()
print('solutions equal?:',dill_t_record.solutions_df.equals(t_record.solutions_df))
print()
print('recs equal?:', dill_t_record == t_record)
print()
dill_t_record.rid = 1
print('recs.rid equal?:', dill_t_record == t_record)
print()
dill_t_record.rid = t_record.rid
dill_t_record.proj_id = 1
print('recs.proj_id equal?:', dill_t_record == t_record)
print()
dill_t_record.proj_id = t_record.proj_id
dill_t_record.iter_id = 1
print('recs.iter_id equal?:', dill_t_record == t_record)
print()
dill_t_record.iter_id = t_record.iter_id
print('recs.reset equal?:', dill_t_record == t_record)

In [ ]:
                 
print()
print('--------------------Test Record with CMTSolutions--------------------------------------------------')
print()

test_cmt_h0 = CMTSolutionHeader(date=datetime.datetime.now(),
                               ename='test_event',
                               tshift=0.0,
                               hdur=0.0,
                               lat_yc=2.71,
                               lon_xc=3.14,
                               depth=-5432,
                               mt=MomentTensor(mw=3.5,strike=120,dip=40,rake=15),
                               eid=7,
                               sid=77)

test_cmt_h1 = test_cmt_h0.copy()
test_cmt_h2 = test_cmt_h0.copy()

print('h0 == h1:',test_cmt_h0 == test_cmt_h1)
print()
print('h0 == h2:',test_cmt_h0 == test_cmt_h2)
print()

print('h0:',test_cmt_h0)
print()
print('h1:',test_cmt_h1)
print()
print('h2:',test_cmt_h2)
print()


flat_grp_stations = sorted(flatten_grouped_headers(l_grp_stations))
print(f'flat_grp_stations[0]:\n{flat_grp_stations[0]}')
print()
#t_record = RecordHeader(solutions_h=[test_cmt_h0],stations_h=flat_grp_stations)
cmt_record = RecordHeader(solutions_h=[test_cmt_h0,test_cmt_h1,test_cmt_h2],stations_h=flatten_grouped_headers(l_grp_stations))
print(f'type(test_cmt_h0)={type(test_cmt_h0)}\n')

print(cmt_record)
print()

cmt_rec_solutions_h = cmt_record.get_solutions_header_list()[0]
cmt_rec_stations_h  = cmt_record.get_stations_header_list()
print(f'station[0]:\n{cmt_rec_stations_h[0]}')
print(f'isinstance(cmt_rec_stations_h[0],ForceSolutionHeader)={isinstance(cmt_rec_stations_h[0],StationHeader)}\n')
print(f'type(flat_grp_stations[0])==type(cmt_rec_stations_h[0]) = {type(flat_grp_stations[0])==type(cmt_rec_stations_h[0])}\n')
print()
print(f'solution[0]:\n{cmt_rec_solutions_h}')
print(f'isinstance(cmt_rec_solutions_h,ForceSolutionHeader)={isinstance(cmt_rec_solutions_h,ForceSolutionHeader)}\n')
print(f'type(test_cmt_h0)==type(cmt_rec_solutions_h) = {type(test_cmt_h0)==type(cmt_rec_solutions_h)}\n')
print()



In [ ]:
f = open(data_out_dir + '/test_cmt_record_header.pickle','wb')
pickle.dump(cmt_record,f)
f.close()

In [ ]:
!ls -lth {data_out_dir + '/test_cmt_record_header.pickle'} 

In [ ]:
f = open(data_out_dir + '/test_cmt_record_header.pickle','rb')
dill_cmt_record = pickle.load(f)
f.close()

print(dill_cmt_record)
print()

'''
dill_nrow = len(dill_cmt_record.stations_df)
cmt_nrow = len(cmt_record.stations_df)
print(f'dill_nrow,cmt_nrow = {dill_nrow},{cmt_nrow}')
#for i in range(dill_nrow):
    #if not dill_cmt_record.stations_df.iloc[i].equals(cmt_record.stations_df.iloc[i]): 
        #print(f'not equal at: {i}')
for i in range(1):
    print(f'dill:\n{dill_cmt_record.stations_df.iloc[7]}')
    print(f'orig:\n{cmt_record.stations_df.iloc[7]}')
''';

dill_nrow = len(dill_cmt_record.solutions_df)
cmt_nrow = len(cmt_record.solutions_df)
print(f'dill_nrow,cmt_nrow = {dill_nrow},{cmt_nrow}')
dill_s = dill_cmt_record.solutions_df.iloc[0]
cmt_s  = cmt_record.solutions_df.iloc[0] 
unzip_dill = list(zip(*dill_s.iteritems()))
unzip_orig = list(zip(*cmt_s.iteritems()))
print(f'dill iteritems:\n{unzip_dill}')
print(f'orig iteritems:\n{unzip_orig}')

print()
print('--------------------------------------------------------')
print()
for i in range(len(unzip_dill)):
    if unzip_dill[i] != unzip_orig[i]:
        print(f'unzip_dill[{i}]\n{unzip_dill[i]}')
        print(f'unzip_orig[{i}]\n{unzip_orig[i]}')
    else:
        print('all clear Mr Magoo')
print()
print('--------------------------------------------------------')
print()
    
print()
print()
print('dill_sol:\n',dill_cmt_record.solutions_df.iloc[0])
print()
print('orig_sol:\n',cmt_record.solutions_df.iloc[0])

print('dill_slice:\n',dill_cmt_record.stations_df.iloc[7])
print()
print('orig_slice:\n',cmt_record.stations_df.iloc[7])
print()
print('to_dict():\n',dill_cmt_record.stations_df.iloc[0].to_dict())
print()
print('stations equal?:',dill_cmt_record.stations_df.equals(cmt_record.stations_df))
#print('stations equal?:',dill_cmt_record.stations_df.equals(dill_cmt_record.stations_df))
print()
print('solutions equal?:',dill_cmt_record.solutions_df.equals(cmt_record.solutions_df))
print()
print('recs equal?:', dill_cmt_record == cmt_record)
print('recs.proj_id equal?:', dill_cmt_record.proj_id == cmt_record.proj_id)
print('recs.rid equal?:', dill_cmt_record.rid == cmt_record.rid)
print('recs.iter_id equal?:', dill_cmt_record.iter_id == cmt_record.iter_id)
print()
dill_cmt_record.rid = 1
print('recs.rid equal?:', dill_cmt_record == cmt_record)
print()
dill_cmt_record.rid = cmt_record.rid
dill_cmt_record.proj_id = 1
print('recs.proj_id equal?:', dill_cmt_record == cmt_record)
print()
dill_cmt_record.proj_id = cmt_record.proj_id
dill_cmt_record.iter_id = 1
print('recs.iter_id equal?:', dill_cmt_record == cmt_record)
print()
dill_cmt_record.iter_id = cmt_record.iter_id
print('recs.reset equal?:', dill_cmt_record == cmt_record)
'''
''';

In [ ]:
def silly_func(x):
    print('Hahwoe!')
    
silly_func = 'Hahwoe!'
    
print(callable(silly_func))

In [ ]:
mys = slice(0,10,2)
print('dir:',dir(mys))
print()
print('dir(index):',dir(mys.indices))
print()
print('start:',mys.start)
print('stop :',mys.stop)
print('step :',mys.step)

In [ ]:
x = np.arange(10)
x[slice(2,10,2)]

In [ ]:
mydf = t_record.stations_df
#row = mydf[0]
h_vals = list(mydf.iloc[1].values)
print('row values :\n',h_vals)
h_keys = list(mydf.iloc[1].index.values)
print('row indicis:\n',h_keys)
print()
ih_keys = list(mydf.index.names)
print('inames:\n',ih_keys)
print()
ih_vals = list(mydf.index.values[1])
print('ivalue:\n',ih_vals)
print()
print('columns:',mydf.columns.values)
print()
#for index, row in mydf.iterrows():
    #print(row)
    
h_dict = dict(zip(ih_keys + h_keys, ih_vals + h_vals))
print('new dict:\n',h_dict)

In [ ]:
dir(StationHeader)

In [ ]:
turk_dict = {'goob':0, 'smoog':1, 'cloeggy':2}
print(dir(turk_dict))

for key in turk_dict:
    print(key)

del turk_dict['goob']
print(turk_dict)

In [ ]:
print(**turk_dict)